In [7]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [8]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (120, 120)
Sub_Model = 5


In [9]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    print('Loading Data...')
    pickle_in = open(Split_Path + 'X.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Split_Path + 'y.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    # Normalizing the features between [0, 1]
    #X = X / 255.0

    return X, y


In [10]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from keras.utils.np_utils import to_categorical
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    # print(model.summary())

    # Compiling the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(X_train, to_categorical(y_train), batch_size=128, epochs=50, verbose=2,
                        validation_data=(X_test, to_categorical(y_test)), shuffle=True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(path + 'Outputs/BRATS/flair_mod_1/Model_' +
               case + '_' + str(count+1) + '.h5')

    return history, model


In [11]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np

    print('Saving Test Accuracy...')

    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')

    for ans in results:
        file.write(str(ans) + ', ')

    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()


In [12]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint

PATH = path + 'Data/BRATS/mod_data1/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/BRATS/flair_mod_1/Results.txt'
workdir = os.listdir(Work_Dir)
if '.DS_Store' in workdir:
    workdir.remove('.DS_Store')
    print('Removed .DS_Store')
# Calling Functions:
for case in tqdm(workdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    # Joining "train" folder with "Cases" path.
    Split_Path = Case_Path + '/train/'
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))

        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0, 1000)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train,
                               y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')
os.system('say "Models are ready"')


Removed .DS_Store


  0%|          | 0/5 [00:00<?, ?it/s]

Case_5
Loading Data...
Shape of X:	 (16592, 120, 120, 1)
Shape of y:	 (16592,)

Case_5_1
Epoch 1/50


2022-11-24 02:07:46.227317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-24 02:07:55.625740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


91/91 - 11s - loss: 3.1827 - accuracy: 0.5040 - val_loss: 0.9207 - val_accuracy: 0.4916 - 11s/epoch - 120ms/step
Epoch 2/50
91/91 - 6s - loss: 1.1409 - accuracy: 0.5269 - val_loss: 0.8629 - val_accuracy: 0.5016 - 6s/epoch - 66ms/step
Epoch 3/50
91/91 - 5s - loss: 0.9435 - accuracy: 0.5312 - val_loss: 0.7774 - val_accuracy: 0.5193 - 5s/epoch - 59ms/step
Epoch 4/50
91/91 - 6s - loss: 0.8540 - accuracy: 0.5452 - val_loss: 0.6848 - val_accuracy: 0.5723 - 6s/epoch - 61ms/step
Epoch 5/50
91/91 - 5s - loss: 0.7846 - accuracy: 0.5574 - val_loss: 0.6927 - val_accuracy: 0.5753 - 5s/epoch - 60ms/step
Epoch 6/50
91/91 - 5s - loss: 0.7634 - accuracy: 0.5569 - val_loss: 0.7275 - val_accuracy: 0.5579 - 5s/epoch - 59ms/step
Epoch 7/50
91/91 - 5s - loss: 0.7396 - accuracy: 0.5617 - val_loss: 0.7566 - val_accuracy: 0.5470 - 5s/epoch - 60ms/step
Epoch 8/50
91/91 - 6s - loss: 0.7229 - accuracy: 0.5722 - val_loss: 0.7138 - val_accuracy: 0.5637 - 6s/epoch - 62ms/step
Epoch 9/50
91/91 - 6s - loss: 0.7070 - a

...

2022-11-24 17:24:47.192362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-24 17:24:54.031400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


92/92 - 8s - loss: 3.2680 - accuracy: 0.5084 - val_loss: 1.0742 - val_accuracy: 0.5170 - 8s/epoch - 91ms/step
Epoch 2/50
92/92 - 6s - loss: 1.0752 - accuracy: 0.5309 - val_loss: 0.7882 - val_accuracy: 0.5394 - 6s/epoch - 65ms/step
Epoch 3/50
92/92 - 6s - loss: 0.9550 - accuracy: 0.5363 - val_loss: 0.7077 - val_accuracy: 0.5529 - 6s/epoch - 65ms/step
Epoch 4/50
92/92 - 6s - loss: 0.8524 - accuracy: 0.5460 - val_loss: 0.7139 - val_accuracy: 0.5358 - 6s/epoch - 66ms/step
Epoch 5/50
92/92 - 6s - loss: 0.7903 - accuracy: 0.5494 - val_loss: 0.7485 - val_accuracy: 0.5209 - 6s/epoch - 67ms/step
Epoch 6/50
92/92 - 6s - loss: 0.7779 - accuracy: 0.5541 - val_loss: 0.7086 - val_accuracy: 0.5354 - 6s/epoch - 68ms/step
Epoch 7/50
92/92 - 6s - loss: 0.7406 - accuracy: 0.5574 - val_loss: 0.6960 - val_accuracy: 0.5479 - 6s/epoch - 66ms/step
Epoch 8/50
92/92 - 6s - loss: 0.7317 - accuracy: 0.5615 - val_loss: 0.7650 - val_accuracy: 0.5189 - 6s/epoch - 67ms/step
Epoch 9/50
92/92 - 6s - loss: 0.7089 - accu

100%|██████████| 5/5 [15:22:07<00:00, 11065.44s/it]  



Saving Test Accuracy...
Done...!!


0